## Segundo Parcial 
### Estadística Aplicada

Cinthia Zaydé Gabrielle García Belmonte

Gpo 44 - 2076329

In [1]:
#Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Librerias especializadas 
from lifelines import CoxPHFitter, KaplanMeierFitter, ExponentialFitter, WeibullFitter, LogNormalFitter
from lifelines.statistics import logrank_test
import seaborn as sns

In [3]:
credit_risk = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/credit_risk.csv')

In [4]:
credit_risk.head()

,saldo_cuenta,meses,historia_crediticia,uso,cantidad,saldo_cuenta_ahorro,años_trabajando,tasa_de_interes,genero_estado_civil,aval,...,activos,edad,planes_de_pago,casa,numero_creditos,situacion_laboral,dependientes_economicos,telefono,extranjero,credito_pagado
0,menor_a_cero,6,no_ha_pagado,TV,1169,NaN,mas_de_7,4,hombre_soltero,NaN,...,bienes_raices,67,NaN,propia,2,gobierno,1,1,1,1
1,cero_a_200,48,pago_algun_credito,TV,5951,menor_100,1_a_4,2,mujer_no_soltera,NaN,...,bienes_raices,22,NaN,propia,1,gobierno,1,0,1,0
2,sin_cuenta,12,no_ha_pagado,colegiaturas,2096,menor_100,4_a_7,2,hombre_soltero,NaN,...,bienes_raices,49,NaN,propia,1,obrero,2,0,1,1
3,menor_a_cero,42,pago_algun_credito,mobiliario,7882,menor_100,4_a_7,2,hombre_soltero,aval,...,seguro_vida,45,NaN,prestada,1,gobierno,2,0,1,1
4,menor_a_cero,24,pago_retrasado,coche_nuevo,4870,menor_100,1_a_4,3,hombre_soltero,NaN,...,sin_activos,53,NaN,prestada,2,gobierno,2,0,1,0


*credito_pagado:* 1: el cliente ha pagado completamente el prestamo, 0: el cliente no ha pagado completamente el prestamo 

*meses:* tiempo en meses que lleva pagar completamente el prestamo

*cantidad:* monto del prestamo en euros

## 1. Análisis Exploratorio

### a) Cuantas variables y cuantas observaciones tienen la base

In [5]:
credit_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   saldo_cuenta             1000 non-null   object
 1   meses                    1000 non-null   int64 
 2   historia_crediticia      1000 non-null   object
 3   uso                      1000 non-null   object
 4   cantidad                 1000 non-null   int64 
 5   saldo_cuenta_ahorro      817 non-null    object
 6   años_trabajando          1000 non-null   object
 7   tasa_de_interes          1000 non-null   int64 
 8   genero_estado_civil      1000 non-null   object
 9   aval                     52 non-null     object
 10  años_actual_domicilio    1000 non-null   int64 
 11  activos                  1000 non-null   object
 12  edad                     1000 non-null   int64 
 13  planes_de_pago           186 non-null    object
 14  casa                     1000 non-null   

La base de datos tiene 1000 observaciones y 21 variables

### b) Cuál es el monto promedio de los créditos otorgados por el banco

In [37]:
print('El monto promedio de los creditos otorgados es de :', credit_risk['cantidad'].mean().round(2), 'euros')

El monto promedio de los creditos otorgados es de : 3271.26 euros


### c) Cuál es el tiempo promedio que se tardaron en pagar el préstamo los que pagaron el préstamo en su totalidad

In [7]:
si_pagados = credit_risk[credit_risk['credito_pagado']==1]

In [8]:
print('El tiempo promedio que se tardaron en pagar el préstamos los que si pagaron el prestamo en su totalidad fue de', si_pagados['meses'].mean().round(0), 'meses')

El tiempo promedio que se tardaron en pagar el préstamos los que si pagaron el prestamo en su totalidad fue de 19.0 meses


### d) Cuál es la tasa de interés promedio otorgada por el banco

In [9]:
print('La tasa promedio de interes otorgada por el banco es de:', credit_risk['tasa_de_interes'].mean())

La tasa promedio de interes otorgada por el banco es de: 2.973


### e) Cuál es el monto promedio y la tasa de interés promedio de los que ya pagaron

In [10]:
print('El monto promedio de los que ya pagaron es de:', si_pagados['cantidad'].mean().round(2), 'euros')

El monto promedio de los que ya pagaron es de: 2985.46 euros


In [11]:
print('La tasa promedio de los que ya pagaron es de:', si_pagados['tasa_de_interes'].mean().round(2))

La tasa promedio de los que ya pagaron es de: 2.92


### f) Cuál es el monto promedio y la tasa de interés promedio de los que no han pagado 

In [12]:
no_pagados = credit_risk[credit_risk['credito_pagado']==0]

In [13]:
print('El monto promedio de los que ya pagaron es de:', no_pagados['cantidad'].mean().round(2), 'euros')

El monto promedio de los que ya pagaron es de: 3938.13 euros


In [14]:
print('La tasa promedio de los que ya pagaron es de:', no_pagados['tasa_de_interes'].mean().round(2))

La tasa promedio de los que ya pagaron es de: 3.1


## 2. Estime un modelo de Kaplan-meier

In [15]:
km1 =KaplanMeierFitter().fit(credit_risk['meses'],credit_risk['credito_pagado'])

### a) cuál es la mediana del tiempo y que significa

In [16]:
print('La mediana de tiempo de supervivencia:', km1.median_survival_time_)

La mediana de tiempo de supervivencia: 24.0


Significa que la mitad de los clientes pagan el credito después de los 24 meses

### b) es mayor/menor que el tiempo promedio del inciso 1c

Es mayor, ya que la mediana del tiempo de supervivencia del modelo KM es de 24 y el tiempo promedio que tardan en apgar es de 19

## 3. Determine si el riesgo de que el préstamo se pague en su totalidad es constante,creciente, decreciente, o tipo log-normal respecto al tiempo

In [17]:
# Calcular los modelos que se podrían utilizar
mexpo = ExponentialFitter().fit(credit_risk['meses'], credit_risk['credito_pagado'])
mweibull = WeibullFitter().fit(credit_risk['meses'], credit_risk['credito_pagado'])
mlogn = LogNormalFitter().fit(credit_risk['meses'], credit_risk['credito_pagado'])

In [18]:
# Determinar cual es el que mejor se ajusta a los datos por medio del AIC (el que tentga menor AIC sera el optimo)
print('Exp', mexpo.AIC_.round(2), ', Weibull', mweibull.AIC_.round(2), ', LogNormal', mlogn.AIC_.round(2))

Exp 6157.19 , Weibull 5900.98 , LogNormal 5813.75


El riesgo de que el prestamo se pague en su totalidad es log-normal respecto al tiempo

## 4. Estimar un modelo de Cox para ver qué factores influyen sobre el riesgo de que el préstamo se pague en su totalidad


### a) A parte de “tasa de interes” y “cantidad”, elige otras cuatro variables que creas que afectan el riesgo del evento

In [19]:
credit_risk.head()

,saldo_cuenta,meses,historia_crediticia,uso,cantidad,saldo_cuenta_ahorro,años_trabajando,tasa_de_interes,genero_estado_civil,aval,...,activos,edad,planes_de_pago,casa,numero_creditos,situacion_laboral,dependientes_economicos,telefono,extranjero,credito_pagado
0,menor_a_cero,6,no_ha_pagado,TV,1169,NaN,mas_de_7,4,hombre_soltero,NaN,...,bienes_raices,67,NaN,propia,2,gobierno,1,1,1,1
1,cero_a_200,48,pago_algun_credito,TV,5951,menor_100,1_a_4,2,mujer_no_soltera,NaN,...,bienes_raices,22,NaN,propia,1,gobierno,1,0,1,0
2,sin_cuenta,12,no_ha_pagado,colegiaturas,2096,menor_100,4_a_7,2,hombre_soltero,NaN,...,bienes_raices,49,NaN,propia,1,obrero,2,0,1,1
3,menor_a_cero,42,pago_algun_credito,mobiliario,7882,menor_100,4_a_7,2,hombre_soltero,aval,...,seguro_vida,45,NaN,prestada,1,gobierno,2,0,1,1
4,menor_a_cero,24,pago_retrasado,coche_nuevo,4870,menor_100,1_a_4,3,hombre_soltero,NaN,...,sin_activos,53,NaN,prestada,2,gobierno,2,0,1,0


In [20]:
# Verificar los nulos en cada variable
credit_risk.isnull().sum()

saldo_cuenta                 0
meses                        0
historia_crediticia          0
uso                          0
cantidad                     0
saldo_cuenta_ahorro        183
años_trabajando              0
tasa_de_interes              0
genero_estado_civil          0
aval                       948
años_actual_domicilio        0
activos                      0
edad                         0
planes_de_pago             814
casa                         0
numero_creditos              0
situacion_laboral            0
dependientes_economicos      0
telefono                     0
extranjero                   0
credito_pagado               0
dtype: int64

Elegí contemplar las siguientes variables: activos, edad, situacion_laboral, dependientes_economicos

### b) Di si cada una de las 6 variables que elegiste, aumentan/disminuyen el riesgo del evento y por qué


**Cantidad** Mientras mayor sea la cantidad a deber disminuye el riesgo de que el prestamo se pague completo

**Tasa de interés** Mientras más alta es la tasa, menor el riesgo de que el prestamo se pague completo

**Activos**Los clientes que poseen algun activo (seguro de vida, bienes raices u otro tipo) tienen mayor riesgo de pagar el prestamo completo que los que estan sin activos

**Edad** Mientras mayor es la edad el riesgo de que el prestamo se pague completo aumenta

**Situación Laboral** Los clientes que tiene situación laboral "gobierno", "obrero" o "administrativo", tienen más riesgo de que el prestamo se pague compelto que los "desempleado"

**Dependientes economicos** Suponiendo que 1 = No tiene dependientes económicos, 2 = Tiene dependientes económicos. Los clientes que tienen dependientes economicos tienen menor riesgo de pagar el prestamo completo que los que no tienen

### c) Realiza las transformaciones adecuadas para utilizar las variables en un modelo de cox


In [22]:
#Tomar solo las 6 variables a utilizar
credit_risk = credit_risk[['meses', 'credito_pagado', 'cantidad', 'tasa_de_interes', 'activos','edad','situacion_laboral','dependientes_economicos']]

In [23]:
#Hacer variable binaria
#1 = Tiene dependientes economicos
#0 = No tiene dependientes economicos
credit_risk['dependientes_economicos'] = credit_risk['dependientes_economicos'] - 1

In [24]:
credit_risk.isnull().sum()

meses                      0
credito_pagado             0
cantidad                   0
tasa_de_interes            0
activos                    0
edad                       0
situacion_laboral          0
dependientes_economicos    0
dtype: int64

No hay nulos

In [25]:
credit_risk.duplicated().sum()

0

No hay duplicados

In [26]:
credit_risk.dtypes

meses                       int64
credito_pagado              int64
cantidad                    int64
tasa_de_interes             int64
activos                    object
edad                        int64
situacion_laboral          object
dependientes_economicos     int64
dtype: object

In [27]:
## crear dummies para las variables categoricas
# Dummies para la columna activos
dummies_activo = pd.get_dummies(credit_risk['activos'], prefix='activos').astype('int64')
dummies_activo.head()

,activos_bienes_raices,activos_otro_tipo,activos_seguro_vida,activos_sin_activos
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,0,0,1,0
4,0,0,0,1


In [28]:
# Eliminamos la columna activo_sin_activo
dummies_activo = dummies_activo[['activos_bienes_raices', 'activos_otro_tipo', 'activos_seguro_vida']]

In [29]:
# Dummies para la columna situacion laboral
dummies_situacion = pd.get_dummies(credit_risk['situacion_laboral'], prefix='situacion_laboral').astype('int64')
dummies_situacion.head()

,situacion_laboral_administrativo,situacion_laboral_desempleado,situacion_laboral_gobierno,situacion_laboral_obrero
0,0,0,1,0
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,0,1,0


In [30]:
# Eliminamos la columna situacion_laboral_desempleado
dummies_situacion = dummies_situacion[['situacion_laboral_administrativo', 'situacion_laboral_gobierno', 'situacion_laboral_obrero']]

In [31]:
credit_risk = pd.concat([credit_risk, dummies_activo, dummies_situacion], axis=1)
credit_risk = credit_risk.drop('activos', axis=1)
credit_risk = credit_risk.drop('situacion_laboral', axis=1)


In [32]:
credit_risk.head()

,meses,credito_pagado,cantidad,tasa_de_interes,edad,dependientes_economicos,activos_bienes_raices,activos_otro_tipo,activos_seguro_vida,situacion_laboral_administrativo,situacion_laboral_gobierno,situacion_laboral_obrero
0,6,1,1169,4,67,0,1,0,0,0,1,0
1,48,0,5951,2,22,0,1,0,0,0,1,0
2,12,1,2096,2,49,1,1,0,0,0,0,1
3,42,1,7882,2,45,1,0,0,1,0,1,0
4,24,0,4870,3,53,1,0,0,0,0,1,0


### d) Estima un modelo de cox con las variables elegidas e interpreta los coeficiente

In [33]:
cph = CoxPHFitter().fit(credit_risk,'meses','credito_pagado')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 1000 total observations, 300 right-censored observations>
             duration col = 'meses'
                event col = 'credito_pagado'
      baseline estimation = breslow
   number of observations = 1000
number of events observed = 700
   partial log-likelihood = -4042.98
         time fit was run = 2023-10-08 02:02:13 UTC

---
                                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                
cantidad                          -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
tasa_de_interes                   -0.41       0.67       0.04            -0.48            -0.33                 0.62                 0.72
edad                               0.02       1.02       0.00             0.01             0.02                 1.01                 1.02
dependientes_economicos            0.08       1.09       0.11            -0.12             0.29                 0.88                 1.34
activos_bienes_raices              0.59       1.81       0.14             0.33             0.86                 1.38                 2.37
activos_otro_tipo                  0.27       1.32       0.13             0.02             0.53                 1.02                 1.71
activos_seguro_vida                0.37       1.45       0.14             0.09             0.65                 1.10                 1.91
situacion_laboral_administrativo   0.48       1.62       0.29            -0.08             1.04                 0.92                 2.83
situacion_laboral_gobierno         0.20       1.22       0.27            -0.33             0.73                 0.72                 2.08
situacion_laboral_obrero           0.36       1.43       0.28            -0.19             0.91                 0.83                 2.48

                                   cmp to      z      p   -log2(p)
covariate                                                         
cantidad                             0.00 -15.29 <0.005     172.86
tasa_de_interes                      0.00 -10.77 <0.005      87.40
edad                                 0.00   4.85 <0.005      19.62
dependientes_economicos              0.00   0.79   0.43       1.22
activos_bienes_raices                0.00   4.33 <0.005      16.02
activos_otro_tipo                    0.00   2.08   0.04       4.73
activos_seguro_vida                  0.00   2.63   0.01       6.87
situacion_laboral_administrativo     0.00   1.69   0.09       3.44
situacion_laboral_gobierno           0.00   0.73   0.46       1.11
situacion_laboral_obrero             0.00   1.28   0.20       2.32
---
Concordance = 0.78
Partial AIC = 8105.96
log-likelihood ratio test = 452.81 on 10 df
-log2(p) of ll-ratio test = 299.90

In [34]:
cph.params_

covariate
cantidad                           -0.000370
tasa_de_interes                    -0.405225
edad                                0.016488
dependientes_economicos             0.084405
activos_bienes_raices               0.594425
activos_otro_tipo                   0.274671
activos_seguro_vida                 0.370368
situacion_laboral_administrativo    0.481515
situacion_laboral_gobierno          0.198619
situacion_laboral_obrero            0.359408
Name: coef, dtype: float64

In [35]:
cph.hazard_ratios_

covariate
cantidad                            0.999630
tasa_de_interes                     0.666827
edad                                1.016625
dependientes_economicos             1.088070
activos_bienes_raices               1.811990
activos_otro_tipo                   1.316097
activos_seguro_vida                 1.448267
situacion_laboral_administrativo    1.618525
situacion_laboral_gobierno          1.219717
situacion_laboral_obrero            1.432481
Name: exp(coef), dtype: float64

**Dependientes economicos, Situacion laboral Administrativo, Situacion laboral Gobierno, Situacion laboral Obrero** Su $P-Value > 0.05 = \alpha$ por lo que **NO** se rechaza $H_0$, esto quiere decir que ninguna de esas variables tiene efecto en el riesgo de que se pague el prestamo completo

**Cantidad** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Mientras mayor es la cantidad del monto del prestamo el riesgo de que se pague el prestamo completo disminuye en un 0.037% 

**tasa_de_interes** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Mientras mayor es la tasa de interés del prestamo el riesgo de que se pague el prestamo completo disminuye en un 0.33% 

**edad** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Por cada año más de edad que tenga el cliente, el riesgo de que se pague el prestamo completo aumenta en un 2%

**activos_bienes_raices** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Los clientes que tienen bienes raices como activos tienen mayor riesgo, un 81% más riesgo, de pagar el prestamo completo que los que no tienen activos

**activos_otro_tipo** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Los clientes que tienen otro tipo de activos tienen mayor riesgo, un 32% más riesgo, de pagar el prestamo completo que los que no tienen activos

**activos_seguro_vida** Como el P-Value es menor a $\alpha = 0.05$ entonces se rechaza $H_0$. Los clientes que tienen seguro de vida como activos tienen mayor riesgo, un 45% más riesgo, de pagar el prestamo completo que los que no tienen activos



## 5. Conclusión

Se realizó un análisis sobre los créditos otorgados a los clientes del Banco de la Ilusión. El análisis se realizó de una base de datos que cuenta con 1000 entradas y 21 diferentes variables, entre las cuales destacan “meses”, “crédito_pagado” y “cantidad”, que nos indican el tiempo en meses que se tarda en pagar el crédito, si el crédito se pago en su totalidad y la cantidad total prestada, respectivamente. 

En el primer análisis exploratorio pudimos observar que en promedio se prestan 3,271.26 euros a los clientes en general, pero el monto prestado promedio para los clientes que si pagaron el préstamo en su totalidad es de 2985.46 euros, lo cual esta por debajo de la media general, esto podría indicar que mientras menor sea el monto más probable es que se pague completo
Por otro lado, la tasa de interés promedio general es de 2.92%, mientras que la tasa de interés promedio de los que no han pagado su préstamo completo es de 3.1%, mayor que la media general, esto puede indicar que mientras más alta es la tasa de interés menor es la probabilidad de que se pague completo el préstamo.

Una vez realizado el análisis exploratorio se seleccionan 6 variables para analizar su relevancia en el riesgo de que el préstamo se pague por completo. Las variables que se cree que incrementaran el riesgo o probabilidad de que el préstamo se pague completo son: activos, edad y situación laboral (donde sus valores no sean “desempleado”)
Los que se espera que disminuyan el riesgo de que el préstamo se pague completo son: tasa de interés, cantidad y dependientes económicos.

Después de realizar el modelo de Cox pudimos confirmar que las variables “edad” y “activos” se comportaron como en nuestra hipótesis, los clientes de mayor edad y los clientes que tienen algún activo tienen mayor probabilidad de pagar el crédito completo por lo que se recomendaría autorizar más créditos a las personas que cumplan estas características.

Por otro lado, mientras mayor sean la cantidad prestada y la tasa de interés, menor es la probabilidad de que el préstamo se pague completo, por lo que se sugiere al banco disminuirlas o tener mayor cuidado al momento de autorizar cantidades grandes.

El resto de las variables no afectan al riesgo de que los prestamos se paguen completos por lo que se puede ignorar.
